# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** See **Lecture 11, Notebook 1** (`11/demo/01_setup_exploration_cleaning.ipynb`), Phase 3 for examples of systematic data cleaning workflows, missing data handling strategies, and outlier detection methods.

---

## Setup

In [8]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

In [11]:
import pandas as pd
import numpy as np
import os

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

# Load dataset
df = pd.read_csv(
    'data/beach_sensors.csv',
    parse_dates=["Measurement Timestamp"],
    index_col="Measurement Timestamp"
)

# Remove duplicates
df = df[~df.index.duplicated(keep='first')]
df = df.drop_duplicates()

# Validate numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].astype(float)

# Handle missing data
df[numeric_cols] = df[numeric_cols].ffill()
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
df[non_numeric_cols] = df[non_numeric_cols].fillna(method='bfill')

# Handle outliers by capping at 1st and 99th percentiles
for col in numeric_cols:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = df[col].clip(lower, upper)

# Save cleaned dataset
df.to_csv('output/q2_cleaned_data.csv', index=False)

/var/folders/c0/4jn3wz693dbg7vh1dl8yg06r0000gp/T/ipykernel_41885/989812686.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[non_numeric_cols] = df[non_numeric_cols].fillna(method='bfill')


In [10]:
import os

file_path = 'output/q2_cleaned_data.csv'
print(os.path.abspath(file_path))
print(os.path.exists(file_path))

/Users/jutang/Desktop/ds217-11-final-dr-justinetang/output/q2_cleaned_data.csv
True


---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data:**
   - Count missing values: `df.isnull().sum()`
   - Choose strategy: drop, forward-fill, impute, etc.
   - For time series: consider `df.ffill()` (forward-fill is appropriate for continuous sensor readings)
   - Implement strategy

2. **Detect and handle outliers:**
   - Use IQR method: `Q1 = df[col].quantile(0.25)`, `Q3 = df[col].quantile(0.75)`, `IQR = Q3 - Q1`
   - Or use z-scores: `z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())`
   - Decide: remove, cap, or transform
   - Document your reasoning

3. **Validate data types:**
   - Check data types: `df.dtypes`
   - Convert as needed: `pd.to_datetime()`, `pd.to_numeric()`
   - Ensure numeric columns are numeric, datetime columns are datetime

4. **Remove duplicates:**
   - Check: `df.duplicated().sum()`
   - Remove: `df.drop_duplicates()`

5. **Document and save:**
   - Write cleaning report to `output/q2_cleaning_report.txt`
   - Save cleaned data to `output/q2_cleaned_data.csv`
   - Save row count to `output/q2_rows_cleaned.txt`

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [25]:
import pandas as pd
import numpy as np
import os

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

# Load original dataset
df = pd.read_csv("data/beach_sensors.csv")

# Copy to cleaned dataset
df_cleaned = df.copy()

# Identify numeric and non-numeric columns
numeric_cols = df_cleaned.select_dtypes(include=['int64', 'float64', 'int32']).columns
non_numeric_cols = df_cleaned.select_dtypes(exclude=['int64', 'float64', 'int32']).columns

# Start report
report_lines = []
report_lines.append("DATA CLEANING REPORT")
report_lines.append("====================\n")

# Rows before cleaning
rows_before = len(df_cleaned)
report_lines.append(f"Rows before cleaning: {rows_before}\n")

# Handle Missing Data
report_lines.append("Missing Data Handling:")

for col in df_cleaned.columns:
    missing_count = df_cleaned[col].isnull().sum()
    if missing_count > 0:
        percent = missing_count / len(df_cleaned) * 100
        # Choose strategy
        if col in numeric_cols:
            strategy = "Forward-fill (time series appropriate)"
            df_cleaned[col] = df_cleaned[col].ffill()  # implement strategy
        else:
            strategy = "Backward-fill for non-numeric columns"
            df_cleaned[col] = df_cleaned[col].bfill()  # implement strategy
        report_lines.append(f"- {col}: {missing_count} missing values ({percent:.1f}%)")
        report_lines.append(f"  Method: {strategy}")
        report_lines.append(f"  Result: All missing values filled")

# Detect and Handle Outliers
report_lines.append("\nOutlier Handling:")

for col in numeric_cols:
    # IQR method
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3*IQR
    upper_bound = Q3 + 3*IQR
    # Count outliers
    outliers_lower = (df_cleaned[col] < lower_bound).sum()
    outliers_upper = (df_cleaned[col] > upper_bound).sum()
    total_outliers = outliers_lower + outliers_upper
    if total_outliers > 0:
        # Handle outliers by capping
        df_cleaned[col] = df_cleaned[col].clip(lower_bound, upper_bound)
        report_lines.append(f"- {col}: Detected {total_outliers} outliers using IQR method (3×IQR)")
        report_lines.append(f"  Method: Capped at bounds [{lower_bound:.2f}, {upper_bound:.2f}]")
        report_lines.append(f"  Result: {total_outliers} values capped")

# Validate Data Types
report_lines.append("\nData Type Conversions:")

# Timestamps
if 'Measurement Timestamp' in df_cleaned.columns:
    df_cleaned['Measurement Timestamp'] = pd.to_datetime(df_cleaned['Measurement Timestamp'])
    report_lines.append("- Measurement Timestamp: Converted to datetime64[ns]")
elif 'Measurement Timestamp Label' in df_cleaned.columns:
    df_cleaned['Measurement Timestamp Label'] = pd.to_datetime(df_cleaned['Measurement Timestamp Label'])
    report_lines.append("- Measurement Timestamp Label: Converted to datetime64[ns]")

# Numeric columns
for col in numeric_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
    report_lines.append(f"- {col}: Converted to float")


# Remove Duplicates
duplicates_removed = df_cleaned.duplicated().sum()
df_cleaned = df_cleaned.drop_duplicates()
report_lines.append(f"\nDuplicates Removed: {duplicates_removed}")

# Rows after cleaning
rows_after = len(df_cleaned)
report_lines.append(f"\nRows after cleaning: {rows_after}")


# Save cleaned dataset and report
df_cleaned.to_csv("output/q2_cleaned_data.csv", index=False)

report_path = "output/q2_cleaning_report.txt"
with open(report_path, "w") as f:
    f.write("\n".join(report_lines))

print(f"Cleaned dataset saved to output/q2_cleaned_data.csv")
print(f"Cleaning report saved to {report_path}")


Cleaned dataset saved to output/q2_cleaned_data.csv
Cleaning report saved to output/q2_cleaning_report.txt


In [26]:
import pandas as pd
import os

# Ensure output folder exists
os.makedirs("output", exist_ok=True)

# Load cleaned data
df_cleaned = pd.read_csv("output/q2_cleaned_data.csv")

# Get number of rows
rows_after_cleaning = len(df_cleaned)

# Save to file
with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(rows_after_cleaning))

print(f"Rows after cleaning ({rows_after_cleaning}) saved to output/q2_rows_cleaned.txt")


Rows after cleaning (195873) saved to output/q2_rows_cleaned.txt
